# Proyek Prediksi Tips Pelayan dengan Regresi Linear
Proyek ini bertujuan untuk membangun model regresi linear sederhana untuk memprediksi besaran tips yang diberikan oleh pelanggan kepada pelayan, berdasarkan berbagai faktor seperti total tagihan, jumlah orang, dan informasi lainnya.

## Import libraries
Langkah pertama adalah mengimpor semua library Python yang dibutuhkan untuk analisis data, pelatihan model, dan visualisasi.

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as pg
import plotly.subplots as make_subplots

## Import Dataset
Mengimpor dataset. Dalam proyek ini, kita akan menggunakan dua dataset: `tips.csv` dan `new_dataset.csv` untuk membandingkan performa model.

In [3]:
# Gunakan dataset 'tips.csv' untuk hasil yang baik
# dataset = pd.read_csv('dataset/tips.csv')

# Atau gunakan 'new_dataset.csv' untuk melihat efek data acak
# dataset = pd.read_csv('dataset/customize_tips_dataset.csv', decimal=',')
# dataset = pd.read_csv('dataset/new_dataset.csv')

# versi dataset yang sudah diperbaiki nilai nya
dataset = pd.read_csv('dataset/fix_tips_dataset.csv')

# sebelum data di cleaning
print("Informasi Dataset:")
print(dataset)
print("\n--- DType Data ---\n")
print(dataset.dtypes)

Informasi Dataset:
      total_bill   tip     sex smoker  day    time  size
0          16.99  1.01  Female     No  Sun  Dinner     2
1          10.34  1.66    Male     No  Sun  Dinner     3
2          21.01  3.50    Male     No  Sun  Dinner     3
3          23.68  3.31    Male     No  Sun  Dinner     2
4          24.59  3.61  Female     No  Sun  Dinner     4
...          ...   ...     ...    ...  ...     ...   ...
1035       14.73  2.20    Male     No  Sat   Lunch     2
1036       10.51  1.25    Male    Yes  Fri   Lunch     3
1037       17.92  3.08    Male    Yes  Fri  Dinner     2
1038       27.20  4.00  Female    Yes  Sat  Dinner     5
1039       22.76  3.00  Female     No  Sat  Dinner     3

[1040 rows x 7 columns]

--- DType Data ---

total_bill    float64
tip           float64
sex            object
smoker         object
day            object
time           object
size            int64
dtype: object


## Cleaning data (opsional)
- ganti semua nilai kolom yg memiliki karakter , (koma)

In [4]:
# hapus spasi header kolom
# dataset.columns = dataset.columns.str.strip()

# dataset['total_bill'] = dataset['total_bill'].str.replace(',', '.', regex=False).astype(float)
# dataset['tip'] = dataset['tip'].str.replace(',', '.', regex=False).astype(float)
# print(dataset)
# # setelah data di cleaning
# print(dataset.dtypes)

# import cleaning data to csv (opsional)
# dataset.to_csv('new_dataset.csv', index=False)

## Visualisasi dataset
Menganalisis dataset untuk mengambil langkah selanjutnya di bagian pengolahan dataset nanti nya

### Menampilkan data pelanggan yang memberikan tips berdasarkan `hari kedatangan nya` dengan diagram scatter (sebaran data)

In [6]:
figure_sc = px.scatter(data_frame=dataset, x='total_bill', y='tip',
                    size='size', color='day', trendline='ols')

figure_pie = px.pie(dataset, values='tip', names='day', hole=0.4)

figure_sc.show()
figure_pie.show()

### Menampilkan data pelanggan berdasarkan `waktu kedatangan nya`

In [7]:
figure_sc = px.scatter(data_frame=dataset, x='total_bill', y='tip',
                    size='size', color='time', trendline='ols')

figure_pie = px.pie(dataset, values='tip', names='time', hole=0.4)

figure_sc.show()
figure_pie.show()

### Menampilkan data pelanggan berdasarkan `gender` 

In [8]:
figure = px.pie(dataset, values='tip', names='sex', hole=0.4)

figure.show()

### Menampilkan data berdasarkan pelanggan `perokok`

In [9]:
figure = px.pie(dataset, values='tip', names='smoker', hole=0.4)

figure.show()

## Data Preprocessing
Model regresi linear membutuhkan input data dalam bentuk numerik. Kita akan melakukan _one-hot encoding_ pada kolom kategorikal (`sex`, `smoker`, `day`, `time`) dan memisahkan fitur (X) serta variabel target (y).

### Transform data kategorikal

In [10]:
dataset['sex'] = dataset['sex'].map({'Female' : 0, 'Male': 1})
dataset['smoker'] = dataset['smoker'].map({'No' : 0, 'Yes': 1})
dataset['time'] = dataset['time'].map({'Lunch' : 0, 'Dinner': 1})
dataset['day'] = dataset['day'].map({'Thur' : 0, 'Fri': 1, 'Sat' : 2, 'Sun' : 3})

print(dataset.head())

   total_bill   tip  sex  smoker  day  time  size
0       16.99  1.01    0       0    3     1     2
1       10.34  1.66    1       0    3     1     3
2       21.01  3.50    1       0    3     1     3
3       23.68  3.31    1       0    3     1     2
4       24.59  3.61    0       0    3     1     4


### Splitting dataset to data training and data testing

In [11]:
# beberapa cara untuk memisahkan variabel independen dan dependen
x = np.array(dataset[['total_bill',
                      'sex',
                      'smoker',
                      'day',
                      'time',
                      'size']
                     ])
# x = dataset.iloc[:, [0,2,3,4,5,6]]
# x = dataset.drop('tip', axis=1)

y = np.array(dataset[['tip']])
# y = dataset.iloc[:, 1]
# y= dataset.drop(['total_bill', 'sex', 'smoker', 'day', 'time', 'size'], axis=1)

# print(x)
# print(y)

### Splitting dataset ke data training dan data testing

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.2,
                                                    random_state=1
                                                    )

## Pelatihan model dan prediksi
Menggunakan data pelatihan (`X_train` dan `y_train`) untuk melatih model Regresi Linear. Setelah model terlatih, kita gunakan untuk membuat prediksi pada data pengujian (`X_test`).

In [13]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [14]:
# fitur yg ingin di prediksi
# total_bill, sex, smoker, day, time, size
features = np.array([[20.50, 1, 1, 2, 1, 4]])
model.predict(features)

array([[2.98965458]])

## Evaluasi Model
Evaluasi performa model dengan metrik umum seperti **Mean Squared Error (MSE)** dan **R-squared (R²)**.

In [15]:
from sklearn.metrics import mean_squared_error, r2_score

# prediksi model pada data testing
y_pred = model.predict(x_test)

# evaluasi model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Hasil Evaluasi Model:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2): {r2:.2f}")

Hasil Evaluasi Model:
Mean Squared Error (MSE): 0.87
Root Mean Squared Error (RMSE): 0.93
R-squared (R2): 0.47


## Visualisasi hasil
Visualisasi membantu kita memahami seberapa baik model memprediksi nilai aktual. Kita akan membuat _scatter plot_ yang membandingkan nilai aktual dengan nilai prediksi dan juga plot residual untuk menganalisis kesalahan model.

In [20]:
# Pastikan array dalam bentuk 1-dimensi untuk visualisasi
y_test_flattened = y_test.ravel()
y_pred_flattened = y_pred.ravel() 
prediction_data = pd.DataFrame({'Actual': y_test_flattened, 'Predicted' : y_pred_flattened })

# Scatter plot
figure = px.scatter(prediction_data, x='Actual', y='Predicted', title='Nilai Actual vs Prediksi')
figure.show()

# Plot residual
residual = y_test_flattened - y_pred_flattened

figure_res = px.scatter(x=y_pred_flattened, y=residual, title='Plot Residual')
figure_res.show()